In [ ]:
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import random

# Function to set seed for reproducibility
def set_seed(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

# Set seed for reproducibility
set_seed(42)

# Load tokenizer
# Subword Tokenization: BERT uses a subword tokenization algorithm called WordPiece.
# This method breaks words into subword units. For example, the word "unbelievable"
# might be broken into "un", "##believ", "##able". The "##" prefix indicates that
# the subword is not the start of a word. This approach allows BERT to handle unknown
# words more gracefully, provide better handling of morphology, and reduce the size
# of the vocabulary needed to cover the majority of tokens in a language.
#
# [CLS]: This token is added to the beginning of every input sequence. BERT uses the
# final hidden state corresponding to this token as the aggregate sequence
# representation for classification tasks.
# [SEP]: This token is used to separate segments in tasks that involve multiple
# sequences (e.g., question answering, where the question and the passage are
#            two separate sequences). It is also used at the end of a single sequence to indicate its end.
# [PAD]: Used to pad sequences to a fixed length necessary for processing in batches.
# [MASK]: Used in BERT's pre-training for the masked language model objective,
# where random tokens are replaced with this token and the model learns to predict the original token.

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#
# Tokenization Process
# Here’s how the BERT tokenizer typically processes text:

# Basic Text Cleaning: The text is cleaned to remove unnecessary characters and split into words.

# WordPiece Tokenization: Words are tokenized into subwords according to the WordPiece algorithm.
# This involves looking up each word in a vocabulary that includes both whole words and subwords.
# If a word is not found in the vocabulary, it is progressively split into smaller subwords until
# subwords that exist in the vocabulary are found.

# Adding Special Tokens: Special tokens like [CLS], [SEP], and [PAD] are added as
# required by the context (e.g., for single or multiple sequences, or to achieve fixed sequence lengths).

# Convert to IDs: The tokens are converted to their corresponding IDs based on the tokenizer’s vocabulary.
# This includes converting all subwords, special tokens, and unrecognized tokens
#  (usually mapped to a special [UNK] token) to their respective indices.

# Segment IDs: For tasks involving multiple sequences, segment IDs (typically 0s and 1s)
# are assigned to differentiate between the first and second sequence.

# Attention Mask: This is generated to tell the model which tokens are actual data and which are padding.
# The attention mask has 1s for actual data tokens and 0s for [PAD] tokens.
#
#
#
# During pre-training, BERT is trained on two main tasks: Masked Language Modeling (MLM) and
# Next Sentence Prediction (NSP). For the NSP task, the [CLS] token representation is used to
# predict whether two segments (sentences) follow each other in the original text. This specific
# training encourages the [CLS] token to encode a representation that captures the relationships and
# context of the entire sequence, making it useful for downstream tasks like classification.

# Example: Load your data here
# For the purpose of this example, let's assume a simple DataFrame with columns 'text' and 'label'
url = "https://raw.githubusercontent.com/clairett/pytorch-sentiment-classification/master/data/SST2/train.tsv"
data = pd.read_csv(url, delimiter='\t', header=None)
texts = data[0].tolist()
labels = data[1].tolist()

# Tokenization and input formatting
encodings = tokenizer(texts, truncation=True, padding=True, max_length=128)

# Convert to PyTorch datasets
inputs = torch.tensor(encodings['input_ids'])
attention_masks = torch.tensor(encodings['attention_mask'])
labels = torch.tensor(labels)

# Create dataset
dataset = TensorDataset(inputs, attention_masks, labels)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Split the data into training and validation sets
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create DataLoaders
batch_size = 16
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)
validation_dataloader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=batch_size)


In [ ]:
print(labels)

tensor(1)


In [ ]:

# Load BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
total_steps = len(train_dataloader) * 4  # Number of training epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training loop
model.train()
for epoch in range(4):  # Loop over the data multiple times
    total_train_loss = 0
    for step, batch in enumerate(train_dataloader):
        b_input_ids, b_input_mask, b_labels = batch
        b_input_ids = b_input_ids.to(device)
        b_input_mask = b_input_mask.to(device)
        b_labels = b_labels.to(device)

        model.zero_grad()

        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        total_train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    # Calculate average loss over the training data.
    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f"Average Train Loss: {avg_train_loss}")

# Validation loop
model.eval()
eval_accuracy = 0
nb_eval_steps = 0

for batch in validation_dataloader:
    b_input_ids, b_input_mask, b_labels = batch
    b_input_ids = b_input_ids.to(device)
    b_input_mask = b_input_mask.to(device)
    b_labels = b_labels.to(device)

    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    logits = outputs.logits
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = accuracy_score(label_ids, np.argmax(logits, axis=1))
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("Accuracy: ", eval_accuracy / nb_eval_steps)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
